In [ ]:
from datasets import load_dataset
import os

#full dataset (600GB of data)
dataset = load_dataset("HuggingFaceFV/finevideo", split="train", num_proc=64)

In [ ]:
dataset[0]['json']

In [ ]:
# Define the category you want to filter by
desired_categories = ['Sports',]# 'Lifestyle', 'Hobbies and Interests', 'Automotive']  # Replace with your desired category

def is_desired_category(sample):
    return sample['json']['content_parent_category'] in desired_categories

filtered_dataset = dataset.filter(is_desired_category, num_proc=32)

In [ ]:
filtered_dataset

In [ ]:
import random 
from tqdm import tqdm

filtered_list = []
save_dir = "/home/ubuntu/temp/large_sports"
os.makedirs(save_dir, exist_ok=True)
mp4_path = os.path.join(save_dir, "mp4s")
os.makedirs(mp4_path, exist_ok=True)

for i, sample in tqdm(enumerate(filtered_dataset), total=len(filtered_dataset)):
    mp4_video_path = f"{mp4_path}/{i}.mp4"
    with open(f"{mp4_video_path}", "wb") as f:
        f.write(sample['mp4'])
    filtered_list.append({"mp4_path": mp4_video_path})

In [ ]:
len(filtered_list)
# save to huggingface dataset
from datasets import Dataset
filtered_dataset = Dataset.from_list(filtered_list)
filtered_dataset.save_to_disk(f"/home/ubuntu/temp/large_sports/large_dataset")

# Save Dataset as Chunks

In [ ]:


def convert_to_seconds(timestamp_str):
        h, m, s = map(float, timestamp_str.split(':'))
        return int(h*3600 + m*60 + s)

filtered_list = []
save_dir = "/home/ubuntu/temp"
os.makedirs(save_dir, exist_ok=True)
mp4_path = os.path.join(save_dir, "mp4s")
os.makedirs(mp4_path, exist_ok=True)

num_chunks = 4
chunk_seconds = 5
for i, sample in tqdm(enumerate(filtered_dataset), total=len(filtered_dataset)):
    mp4_video_path = f"{mp4_path}/{i}.mp4"

    chunks = []
    all_activities = []
    for activity in sample['json']['content_metadata']['scenes']:
        all_activities += activity['activities']
    timestamps_start = []
    timestamps_end = []
    for act in all_activities:
        # convert from string to seconds '00:02:01.000' -> 121
        try:
            timestamp_start = convert_to_seconds(act['timestamp']['start_timestamp'])
        except:
            timestamp_start = 0
        try:
            timestamp_end = convert_to_seconds(act['timestamp']['end_timestamp'])
        except:
            timestamp_end = timestamp_start + 5
        timestamps_start.append(timestamp_start)
        timestamps_end.append(timestamp_end)

    # 0. Filter out if few activities
    # print(len(timestamps_start), num_chunks, timestamps_start[num_chunks-1], chunk_seconds*num_chunks)
    if(len(timestamps_start) < num_chunks):
        continue
    else:
        if(timestamps_start[num_chunks-1] > chunk_seconds*num_chunks):
            continue
    # filter out if duration is less than num_chunks*chunk_seconds
    if(timestamps_start[-1] - timestamps_start[0] < num_chunks*chunk_seconds):
        continue
    # 1. Create Chunks
    curr_visible_activities = []
    activity_idx = 0
    for j in range(num_chunks):
        chunk = {
            'interval': (j*chunk_seconds, (j+1)*chunk_seconds),
        }
        for idx, act in enumerate(all_activities):
            if(idx >= activity_idx):
                if(timestamps_start[idx] < (j+1)*chunk_seconds):
                    curr_visible_activities.append(all_activities[activity_idx])
                    activity_idx += 1
        #random sample from visible activities
        if(len(curr_visible_activities) == 0):
            chunk['activity'] = {"start": timestamps_start[0], 
                                 "end": timestamps_end[0], 
                                 "description": all_activities[0]['description']}
        else:
            random_idx = random.randint(0, len(curr_visible_activities)-1)
            chunk['activity'] = {"start": timestamps_start[random_idx],
                                 "end": timestamps_end[random_idx], 
                                 "description": curr_visible_activities[random_idx]['description']}
        chunks.append(chunk)


    # C. Video mp4 save
    with open(f"{mp4_video_path}", "wb") as f:
        f.write(sample['mp4'])
    
    filtered_list.append({"chunks": chunks, "mp4_path": mp4_video_path})

In [ ]:
len(filtered_list)
# save to huggingface dataset
from datasets import Dataset
filtered_dataset = Dataset.from_list(filtered_list)
filtered_dataset.save_to_disk(f"/home/ubuntu/temp/{desired_category}_dataset")

In [ ]:
filtered_dataset[0]

In [ ]:
filtered_dataset[0]['json']['content_metadata']['scenes'][0]['activities']

In [ ]:
chunks

In [ ]:
timestamps_start

In [ ]:
chunks

In [ ]:
filtered_dataset[0]['json'].keys()

In [ ]:
all_activities

In [ ]:
for i, sample in enumerate(filtered_dataset):
    with open(f"{mp4_path}/{i}.mp4", "wb") as f:
        f.write(sample['mp4'])

    chunks = {}
    all_activities = []
    for activity in sample['json']['content_metadata']['scenes']:
        all_activities += activity['activities']

    acts = all_activities
    timestamps_start = []
    timestamps_end = []
    def convert_to_seconds(timestamp_str):
        h, m, s = map(float, timestamp_str.split(':'))
        return int(h*3600 + m*60 + s)
    for act in acts:
        # convert from string to seconds '00:02:01.000' -> 121
        timestamp_start = convert_to_seconds(act['timestamp']['start_timestamp'])
        timestamp_end = convert_to_seconds(act['timestamp']['end_timestamp'])
        
        timestamps_start.append(timestamp_start)
        timestamps_end.append(timestamp_end)

    import matplotlib.pyplot as plt
    plt.figure()
    plt.plot(timestamps_start, timestamps_start, 'ro')
    plt.plot(timestamps_start, timestamps_end, 'bo')
    plt.show()

    if(i>10):
        break